In [1]:
import cantera as ct 
import cantera.ck2cti
from rmgpy import chemkin
import rmgpy.chemkin
from rmgpy import kinetics
import numpy as np
import subprocess
import csv
import copy
import os

Function: Load_models

In [2]:
def load_models(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2):
    
    full_path_1 = full_path_1
    inp_path_1 = full_path_1 + chemkin_1
    dict_path_1 = full_path_1 + dictionary_1
    model_1_species, model_1_reactions = chemkin.load_chemkin_file(inp_path_1, dict_path_1)
    
    full_path_2 = full_path_2
    inp_path_2 = full_path_2 + chemkin_2
    dict_path_2 = full_path_2 + dictionary_2
    model_2_species, model_2_reactions = chemkin.load_chemkin_file(inp_path_2, dict_path_2)

    
    different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq__2, model_1_species, model_1_reactions, model_2_species, model_2_reactions  = execute(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2)
    
   # execute_flip(different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq__2, model_1_species, model_1_reactions, model_2_species, model_2_reactions)
    
    return different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq__2, model_1_species, model_1_reactions, model_2_species, model_2_reactions 


Functions: Compare Reactions

In [3]:
def compare_rxn_equations(model_1_reactions, model_2_reactions):
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (model_1_reactions, model_2_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    for reactionList, reaction in to_remove:
        reactionList.remove(reaction)
    print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    unique_reactions_1 = []
    unique_reactions_2 = []
    copy_of_model_2_reactions = model_2_reactions.copy() 
    copy_of_model_1_reactions = model_1_reactions.copy() 

    
    for rxn_2 in copy_of_model_2_reactions:
        for rxn_1 in copy_of_model_1_reactions:  # make a copy so you don't remove from the list you are iterating over
            if rxn_2.is_isomorphic(rxn_1):
                common_reactions.append([rxn_2, rxn_1])
                #common_reactions_equations.append([str(rxnN), str(rxnD)])
                # Remove reaction 2 from being chosen a second time.
                # Let each reaction only appear only once in the diff comparison.
                # Otherwise this miscounts number of reactions in model 2.
                copy_of_model_1_reactions.remove(rxn_1)
 
    
    for rxn_2 in copy_of_model_2_reactions:
        for r_2, r_1 in common_reactions:
            if rxn_2.is_isomorphic(r_2):
                if rxn_2 is not r_2:
                    print(str(rxn_2))
                break
        else:
            unique_reactions_2.append(rxn_2)
    for rxn_1 in copy_of_model_1_reactions:
        for r_2, r_1 in common_reactions:
            if rxn_1.is_isomorphic(r_1):
                break
        else:
            unique_reactions_1.append(rxn_1)
            
    different_kinetics_reactions = []
            
    for r_2, r_1 in common_reactions: 
        if (r_2.kinetics.is_identical_to(r_1.kinetics)==False) and (r_2.kinetics.is_similar_to(r_1.kinetics)==False):
            different_kinetics_reactions.append(r_1)
            
    return different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2
    
    
    

def details(different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2):
    
    '''
    Takes rmg objects and extracts readable info (equations, indexes, etc)
    '''
    
    diff_kin_equations = [str(i) for i in different_kinetics_reactions]
    comm_equations = [str(i) for i in common_reactions]
    
    uniq_rxns_eq_1 = [str(i) for i in unique_reactions_1]

    uniq_rxns_eq_2 = [str(i) for i in unique_reactions_2]
    
    return diff_kin_equations, comm_equations, uniq_rxns_eq_1, uniq_rxns_eq_2 
    
    
    
    
    
    
    
def execute(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2): 
    '''
    loads in chemkin files and runs commands to execute the file. Model 1 should be larger. 
    '''
    full_path_2 = full_path_2
    inp_path_2 = full_path_2 + chemkin_2
    dict_path_2 = full_path_2 + dictionary_2
    model_2_species, model_2_reactions = chemkin.load_chemkin_file(inp_path_2, dict_path_2)

    full_path_1 = full_path_1
    inp_path_1 = full_path_1 + chemkin_1
    dict_path_1 = full_path_1 + dictionary_1
    model_1_species, model_1_reactions = chemkin.load_chemkin_file(inp_path_1, dict_path_1)
    
    different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2 = compare_rxn_equations(model_1_reactions, model_2_reactions)
    diff_kin_equations, comm_equations, uniq_rxns_eq_1, uniq_rxns_eq_2  = details(different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2)
    return different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq_2, model_1_species, model_1_reactions, model_2_species, model_2_reactions 

    

Functions: One_Flip_120_140

In [4]:
def find_new_species(model_1_species, model_2_species):
    '''
    find the new species added in to the larger model.
    '''
    list_of_uniq_species = []
    lbls_2 = []
    for i in range(len(model_2_species)):
        lbl_2 = model_2_species[i].label
        lbls_2.append(lbl_2)
    for j in range(len(model_1_species)):
        lbl_1 = model_1_species[j].label
        if lbl_1 not in lbls_2: 
            print(f'The species added in was: {lbl_1}')
            uniq_spec = model_1_species[j]
            list_of_uniq_species.append(uniq_spec)
    if len(list_of_uniq_species)>1: 
        uniq_species = list_of_uniq_species
    else: 
        uniq_species = list_of_uniq_species[0]
    return uniq_species

def add_in_new_spec(model_1_species, model_2_species):
    """
    add in the new species of the larger model to the smaller model. 
    """
    #larger_model= model_1, smaller_model = model_2
    uniq_species = find_new_species(model_1_species, model_2_species)

    sm_species = [copy.deepcopy(spec) for spec in model_2_species]
    length_before = len(sm_species)
    sm_species.append(uniq_species)
    length_after = len(sm_species)
    
    print(f'Species list extended from {length_before} species to {length_after} species.')
    return sm_species

# def 



# def execute_flip(common_reactions, unique_reactions_1, unique_reactions_2, model_1_species, model_1_reactions, model_2_species, model_2_reactions):
    
    
    
    
    
    
    
    
    
    #########need to add execute_flip to load models###############

    


In [19]:
    '''
    Add in the new reactions to the small model one at a time.  so we can add the new reactions to the model that involve the new species
    '''
    #remember, larger_model= model_1, smaller_model = model_2
    new_model_2_species = add_in_new_spec(model_1_species, model_2_species)
    assert len(common_reactions)== len(model_2_reactions) #make sure that common reactions includes all of the ones in smaller model
        
    #convert each reaction in unique_reactions_1 to use the same species labeling as the smaller model
    ######### still need this! for some reason, the last two don't map correctly
    _1_to_2_mapping ={}
    for i, sp_1 in enumerate(model_1_species):
        for j, sp_2 in enumerate(new_model_2_species):
            if sp_1.is_isomorphic(sp_2):
                _1_to_2_mapping[i] = j 
                break
                
   #for the reactants and products in unique_reactions_1, change them to the model_2 labeling
    converted_rxns_to_add = [] 
    sm_rxns = [copy.deepcopy(rxn) for rxn in model_2_reactions] #this is the list that we will add the unique reactions to 
    
    #convert the entire list of unique_reactions_1
    #absolutely did not need to do this bit. Although last two mappings in 1_to_2_mappings are switched, their labels are still the same. We'll keep it anyways
    for i in range(len(unique_reactions_1)):
        rxn_to_be_added = copy.deepcopy(unique_reactions_1[i])    
        reactants = []
        for reactant in unique_reactions_1[i].reactants:
            try:
                spc_index_in_1 = model_1_species.index(reactant)
                reactants.append(new_model_2_species[_1_to_2_mapping[spc_index_in_1]])
            except ValueError:
                if reactant in model_1_species:
                    reactants.append(reactant)
        rxn_to_be_added.reactants = reactants

        products = []
        for product in unique_reactions_1[i].products:
            try:
                spc_index_in_1 = model_1_species.index(product)
                products.append(new_model_2_species[_1_to_2_mapping[spc_index_in_1]])
            except ValueError:
                if product in model_1_species:
                    products.append(product)
        rxn_to_be_added.products = products

        
        converted_rxns_to_add.append(rxn_to_be_added)

      #check if we needed to convert at all                           
#     for i, j in zip(converted_rxns_to_add, unique_reactions_1):
#         if str(i) != str(j):
#             print(str(i), "\t", str(j)
    
    #visual check is species in two models are the same after updating the species in the smaller model 
#    for i in range(len(new_model_2_species)):
 #       print(str(new_model_2_species[i]), str(model_1_species[i]))
        
    model_dir = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/NK_120_to_140/one_flip_models'
    transport_path = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/Nora/2_BTP/FFCM_seed/2_BTP_seed/chemkin/copies/tran.dat'
    
    reactions_to_be_used = [rxn for rxn in model_2_reactions]
    print(f'Smaller model originally had {len(reactions_to_be_used)} reactions.')
    print('Now generating models with the following number of reactions:')
    for i in range(len(converted_rxns_to_add)):
        print(i)
        print(f"converted_rxns_to_add[i].index: {converted_rxns_to_add[i].index}")
        reactions_to_be_used.append(converted_rxns_to_add[i])
        
        #save the model
        
        chemkin_file = os.path.join(model_dir, f'chem_{converted_rxns_to_add[i].index}.inp')
        rmgpy.chemkin.save_chemkin_file(chemkin_file, new_model_2_species, reactions_to_be_used, verbose=True, check_for_duplicates=True)
#         os.system('source activate cantera_env')
#         os.system(f'ck2cti --input={chemkin_file} --transport={transport_path}')
        #subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport_path}', f'--output={model_dir}/chem_{converted_rxns_to_add[i].index}.cti'])
        print(f"Length of reactions_to_be_used: {len(reactions_to_be_used)}")
        print('\n')
        
    try: 
        assert len(reactions_to_be_used) == len(model_1_reactions) 
        print("Successfully added in all additional reactions.")
    except AssertionError:
        print("Could not add in all additional reactions.")#make sure we're adding everything in correctly
        


The species added in was: C3H3
Species list extended from 129 species to 130 species.
Smaller model originally had 1256 reactions.
Now generating models with the following number of reactions:
0
converted_rxns_to_add[i].index: 1270
Length of reactions_to_be_used: 1257


1
converted_rxns_to_add[i].index: 1271
Length of reactions_to_be_used: 1258


2
converted_rxns_to_add[i].index: 1272
Length of reactions_to_be_used: 1259


3
converted_rxns_to_add[i].index: 1273
Length of reactions_to_be_used: 1260


4
converted_rxns_to_add[i].index: 1274
Length of reactions_to_be_used: 1261


5
converted_rxns_to_add[i].index: 1275
Length of reactions_to_be_used: 1262


6
converted_rxns_to_add[i].index: 1276
Length of reactions_to_be_used: 1263


7
converted_rxns_to_add[i].index: 1277
Length of reactions_to_be_used: 1264


8
converted_rxns_to_add[i].index: 1278
Length of reactions_to_be_used: 1265


9
converted_rxns_to_add[i].index: 1279
Length of reactions_to_be_used: 1266


10
converted_rxns_to_add[i]

In [10]:
print(len(reactions_to_be_used))

print(len(model_1_reactions))

print(len(converted_rxns_to_add))
# #for i in range(len(model_1_reactions)):
# #    print(model_1_reactions[i].index)
# s = [x for x in model_2_reactions]
# for i in range(len(converted_rxns_to_add)):
#     s.append(converted_rxns_to_add[i])
# for i in s: 
#     print(i.index)
    
    
    #indices are the same, so we can just 

1300
1300
44


In [32]:

for i in range(len(model_1_reactions)): #these are all the new reactions in model_1 that are not in model_2
    print(str(model_1_reactions[i].index))

print('\n\n\n')

for i in range(len(model_1_reactions)):
    print(str(model_1_reactions[i].index))

for i in range(len(converted_rxns_to_add)): #these are all the reactions that will be added to model_2
    print(str(converted_rxns_to_add[i].index))
    
    
    #are these the same??? YES, so why is the indexing off? 
    


1300

In [31]:
for i in range(len(model_1_reactions)): #these are all the new reactions in model_1 that are not in model_2
    print(str(model_1_reactions[i].index))

1
2
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
21
23
24
25
26
28
29
30
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
28

Run 

In [7]:
different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq_2, model_1_species, model_1_reactions, model_2_species, model_2_reactions  = load_models(
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/models/Nora/2_BTP_seed/chemkin/",
    "copies/copy_chem_130.inp",
    "species_dictionary.txt",
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/models/Nora/2_BTP_seed/chemkin/",
    "copies/copy_chem_129.inp",
    "species_dictionary.txt"
    )

#WHEN COMPARING TO LAST SCRIPT:
#load_models(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2)
#execute(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2) 
#therefore, D = 1 = 130, N = 2 = 120

Following reactions removed because they have unknown species: []
CH2CHO(35) <=> CO(15) + CH3(19)
CO(15) + CH3(19) <=> HCO(17) + CH2(T)(18)
O2(157) + 2-BTP(1) <=> O2(4) + 2-BTP(1)
CF3CCH(84) + 2-BTP(1) <=> CF3CCH(84) + S(164)
CF3CCH(84) + 2-BTP(1) <=> CF3CCH(84) + S(164)
CF3CCH(84) + 2-BTP(1) <=> CF3CCH(84) + S(164)
S(125) + 2-BTP(1) <=> S(125) + S(164)
S(125) + 2-BTP(1) <=> S(125) + S(164)
S(125) + 2-BTP(1) <=> S(125) + S(164)
CH2O(20) + 2-BTP(1) <=> CH2O(20) + S(164)
CH2O(20) + 2-BTP(1) <=> CH2O(20) + S(164)
CH2O(20) + 2-BTP(1) <=> CH2O(20) + S(164)
O2(4) + S(164) <=> O2(157) + 2-BTP(1)
O2(4) + S(164) <=> O2(4) + 2-BTP(1)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
BR(90) + 2-BTP(1) <=> BR(90) + S(164)
BR(90) + 2-BTP(1) <=> S(144)
C2H4(30) + 2-BTP(1) <=> C2H4(30) + S(164)
O2(157) + CF3CCH(84) <=> O2(4) + CF3CCH

Trying it out

In [ ]:
#compare the unique reactions 
print(len(unique_reactions_1))
for i in range(len(unique_reactions_1)): 
    print(str(unique_reactions_1[i]))
    print(type(unique_reactions_1[i].kinetics))
    print(unique_reactions_1[i].kinetics.is_pressure_dependent())
    print('\n')
    

In [ ]:
#find the species added in 
lbls_2 = []
for i in range(len(model_2_species)):
    lbl_2 = model_2_species[i].label
    lbls_2.append(lbl_2)
for j in range(len(model_1_species)):
    lbl_1 = model_1_species[j].label
    if lbl_1 not in lbls_2: 
        print(f'The species added in was: {lbl_1}')
        uniq_spec = model_1_species[j]
        
#how many of the new equations involve the new species
rxns_with_C3H3 = []
for i in range(len(unique_reactions_1)):
    rxn_spec = []
    for x in unique_reactions_1[i].reactants:
        rxn_spec.append(x)
    for j in unique_reactions_1[i].products:
        rxn_spec.append(j)

    for spc in rxn_spec:
        if spc is uniq_spec: 
            #print(unique_reactions_1[i])
            rxns_with_C3H3.append(unique_reactions_1[i])
    
print(len(rxns_with_C3H3))

In [ ]:
#pressure-dependent?
for i in range(len(unique_reactions_1)):
    print(unique_reactions_1[i].kinetics.is_pressure_dependent())

# Report: 

130 model (1): has 1306 reactions (1306 from RMG.log, 1300 from load.chemkin)

129 model (2): has 1262 reactions (1262 from RMG.log, 1256 from load.chemkin)

- from 129 to 130 model, the species added in was: [CH]=C=C(6822)
- difference of 44 reactions, only 10 of which are pressure dependent
    - of the 44 reactions, 40 involve [CH]=C=C(6822)
    



